# 决策树
>目标：找到一个衡量标准，来计算不同特征进行分支选择后的分类情况，并且将最好的那个当作根节点，以此类推

__衡量标准：熵__

熵是表示随机变量不确定性的度量，公式为：
$$H(X)=-\sum_i p_i*log(p_i)$$
其中，熵越大，不确定性越高。
1. 当$p=0 or 1$的时候，$H(p)=0$，完全没有不确定性性
2. 当$p=0.5$的时候，$H(p)$最大，此时变量的不确定性最大

在分类任务中，期望通过分类后熵值的减少来评判
### 决策树算法
__ID3：信息增益__

表示特征X使得类Y的不确定性减少的程度（希望分类后的结果是同类在一起）

不适合解决稀疏的特征

__C4.5:信息增益率__

__GINI系数：__
和熵类似
$$Gini(p)=\sum_{k=1}^K p_k(1-p_k) = 1-\sum_{k=1}^Kp_k^2$$

### 连续值的处理
对连续值进行排序之后，选择可能的分界点（离散化的过程），使用贪心算法寻找效果最好的方法

### 剪枝策略
为了防止过拟合，进行剪枝

__预剪枝__：限制深度、叶子结点个数、叶子节点样本数、信息增益量等

__后剪枝__: 建立完决策树之后，通过一定的标准衡量$$C_{\alpha}(T)=C(T)+\alpha\cdot\left|T_{leaf}\right|$$(叶子结点越多，损失越大)

$\alpha$是控制计算损失的参数，$\alpha$越大，则越不容易过拟合

### 回归问题
回归问题不能使用熵来衡量，可以使用方差，衡量划分的好坏。最后回归值等于归类之后的平均数

# 代码实现

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import operator
import pickle
from matplotlib.font_manager import FontProperties

# 先创建一个数据集
def creatDataset():
    dataSet = [[0,0,0,0,'no'],
               [0,0,0,1,'no'],
               [0,1,0,1,'yes'],
               [0,1,1,0,'yes'],
               [0,0,0,0,'no'],
               [1,0,0,0,'no'],
               [1,0,0,1,'no'],
               [1,1,1,1,'yes'],
               [1,0,1,2,'yes'],
               [1,0,1,2,'yes'],
               [2,0,1,2,'yes'],
               [2,0,1,1,'yes'],
               [2,1,0,1,'yes'],
               [2,1,0,2,'yes'],
               [2,0,0,0,'no']]
    labels = ['F1-AGE','F2-WORK','F3-HOME','F4-LOAN']
    return dataSet,labels


In [ ]:
def creatTree(dataset, labels,featLabels):
    # 递归生成决策树
    classList = [sample[-1] for sample in dataset]
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    if len(dataset[0])==1:
        return majorityCnt(classList)
    bestFeat = chooseBestFeatureToSplit(dataset)
    bestFeatLable = labels[bestFeat]
    featLabels.append(bestFeatLable)
    myTree = {bestFeatLable:{}}
    del labels[bestFeat]
    featValue = [sample[bestFeat] for sample in dataset]
    uniqueVals = set(featValue)
    for value in uniqueVals:
        sublabels = labels[:]
        myTree[bestFeatLable][value]=creatTree(splitDataSet(dataset, bestFeat, value),sublabels)

    return myTree

def majorityCnt(classList):
    classCount={}
    for vote in classList:
        if vote not in classCount.keys():
            classCount[vote]=0
        classCount[vote]+=1
    sorted_classCount = sorted(classCount.items(),key=operator.itemgetter(1),reverse=True)
    return sorted_classCount[0][0]


# 创建决策树类

In [5]:
import numpy as np
class DecisionNode:
    def __init__(self, feature_index=None, threshold=None, 
                 value=None, left=None, right=None):
        self.feature_index = feature_index   # 特征索引
        self.threshold = threshold           # 分割阈值
        self.value = value                   # 结点值
        self.left = left                     # 左子结点
        self.right = right                   # 右子结点

class DecisionTreeClassifier:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None
    def _gini_index(self, y):
        classes = np.unique(y)
        gini = 0.0
        for cls in classes:
            p = np.sum(y == cls) / len(y)
            gini += p * (1 - p)
        return gini
    def _best_split(self, X, y):
        best_gini = float('inf')
        best_feature_index = 0
        best_threshold = 0
        for feature_index in range(X.shape[1]):
            for threshold in np.unique(X[:, feature_index]):
                left_mask = X[:, feature_index] <= threshold
                right_mask = X[:, feature_index] > threshold
                gini = (len(y[left_mask]) * self._gini_index(y[left_mask])
                        + len(y[right_mask]) * self._gini_index(y[right_mask])) / len(y)
                if gini < best_gini:
                    best_gini = gini
                    best_feature_index = feature_index
                    best_threshold = threshold
        return best_feature_index, best_threshold
    def _build_tree(self, X, y, depth):
        if depth == self.max_depth or len(np.unique(y)) == 1:
            return DecisionNode(value=np.argmax(np.bincount(y)))
        feature_index, threshold = self._best_split(X, y)
        left_mask = X[:, feature_index] <= threshold
        right_mask = X[:, feature_index] > threshold
        left = self._build_tree(X[left_mask], y[left_mask], depth + 1)
        right = self._build_tree(X[right_mask], y[right_mask], depth + 1)
        return DecisionNode(feature_index, threshold, left=left, right=right)
    def fit(self, X, y):
        self.tree = self._build_tree(X, y, 0)
    def predict(self, X):
        results = []
        for sample in X:
            node = self.tree
            while node.left:
                if sample[node.feature_index] <= node.threshold:
                    node = node.left
                else:
                    node = node.right
            results.append(node.value)
        return results

### 使用鸢尾花数据集进行实验

In [13]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# 加载数据集
data = load_iris()
X = data.data
y = data.target
# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# 创建决策树分类器对象
clf = DecisionTreeClassifier(max_depth=3)
# 拟合训练数据
clf.fit(X_train, y_train)
# 预测测试数据
y_pred = clf.predict(X_test)
# 计算准确率
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy*100,"%")

Accuracy: 100.0 %
